# Import packages

In [ ]:
import pandas as pd
from scipy import stats
import statistics  

# Import monthly access portfolio returns

In [ ]:
df = pd.read_excel("Aggregate Returns.xlsx")
# Convert dates to string
df['DATE'] = df['DATE'].astype(str)
# Get access returns
df = df.iloc[:, [0, 7, 8, 9, 10]]

# Calculate the anual Sharpe ratio for each portfolio

In [ ]:
test_years = ['2016', '2017', '2018', '2019', '2020', '2021']

# Pre-define dataframe
Sharpe_results = pd.DataFrame(
    {"Year":                [None, None, None, None, None, None],
     "MLP":                 [None, None, None, None, None, None],
     "SRNN":                [None, None, None, None, None, None],
     "EWP":                 [None, None, None, None, None, None],
    })

for year,row in zip(test_years, range(6)):
    full_window = df[df['DATE'].str.contains(year)]
    MLP = full_window.iloc[:, 1]
    SRNN = full_window.iloc[:, 2]
    EWP = full_window.iloc[:, 3]
    sixfour = full_window.iloc[:, 4]
    
    # Calculate Sharpe ratios
    MLP_Sharpe = statistics.mean(MLP) / statistics.stdev(MLP) * statistics.sqrt(12)
    SRNN_Sharpe = statistics.mean(SRNN) / statistics.stdev(SRNN) * statistics.sqrt(12)
    EWP_Sharpe = statistics.mean(EWP) / statistics.stdev(EWP) * statistics.sqrt(12)
    
    # Allocate to data frame
    Sharpe_results.at[row, "Year"] = year
    Sharpe_results.at[row, "MLP"] = MLP_Sharpe
    Sharpe_results.at[row, "SRNN"] = SRNN_Sharpe
    Sharpe_results.at[row, "EWP"] = EWP_Sharpe
    
Sharpe_results.to_csv("Sharpe results.csv")

# Performone sided t-tests

In [ ]:
MLP = Sharpe_results["MLP"]
SRNN = Sharpe_results["SRNN"]
EWP = Sharpe_results["EWP"]
EWP = Sharpe_results["60/40"]

alpha = 0.05

# See whether MLP significantly outperforms EWP (one sided t-test)
t_stat, p_value = stats.ttest_ind(MLP, EWP, alternative='greater')

print(t_stat, p_value)

if p_value < alpha:
    print("Reject the null hypothesis. There is evidence that the mean Sharpe for MLP  is greater than the EW portfolio.")
else:
    print("Fail to reject the null hypothesis. There is insufficient evidence that the mean Sharpe for MLP is greater than the EW portfolio.")

# See whether SRNN significantly outperforms EWP (one sided t-test)
t_stat, p_value = stats.ttest_ind(SRNN, EWP, alternative='greater')

print(t_stat, p_value)

if p_value < alpha:
    print("Reject the null hypothesis. There is evidence that the mean Sharpe for SRNN  is greater than EWP.")
else:
    print("Fail to reject the null hypothesis. There is insufficient evidence that the mean Sharpe for SRNN is greater than the EW portfolio.")